# DSO560 Homework 4 

## Xiangyu Huang



1. Identify **three pairs of documents** in the McDonalds review dataset that have over 0.85 cosine similarity using average token word2vec embeddings from spacy.

2. Using the `SMS_test` and `SMS_train` datasets, build a classification model (you can simply use the `sklearn.linear_model.LogisticRegression` model used. Please attempt at least two of the vectorization techniques below:
* `CountVectorization`
* `TfIdfVectorization`
* `word2vec` spacy document-level vectors

Make sure you perform the following:
* use train/test split
* use proper model evaluation metrics
* text preprocessing (regex, stemming/lemmatization, stopword removal, grouping entities, etc.)

A discussion of the following:
* **What techniques** you tried to improve the performance of your model.
* What you would try to do, given more time, that would improve the performance of your model.
* Provide an example of two **error cases** - a false positive and a false negative - that your model got wrong, and why the model did not predict the correct answer.

  **Part I**

In [22]:
import pandas as pd
import numpy as np
import math
import nltk
import re
import spacy
import matplotlib.pyplot as plt 
import spacy

from itertools import product
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
from sklearn.model_selection import train_test_split

from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from spacy.matcher import Matcher 
from spacy.tokens import Span
from scipy.spatial.distance import cosine
from itertools import product
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English

In [23]:
nltk.download('punkt') 
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/huangsky/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/huangsky/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/huangsky/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [24]:
mcd_review = pd.read_csv("mcdonalds-yelp-negative-reviews.csv", encoding = 'latin-1' )

In [25]:
mcd_review['review'] = mcd_review['review'].str.lower()

mcd_review['review'] = mcd_review['review'].str.replace(r"\b(\w+)?burgers?\b|\b(\w+)?sandwich(\w+)?\b|\b(?:ham|chesse|steak|chicken|double|triple)s?(?:\s)(?:burger|bun|sandwich)s?\b" ,
                                                        'burger')

mcd_review['review'] = mcd_review['review'].str.replace(r"\bmcds?\b|\bmcdonald's\b|\bmacd\b|\bmc(\w+)?\b|\b(\w+)?mcdonald?(\w+)?\b" , 
                                                        'macdonald')
mcd_review['review'] = mcd_review['review'].str.replace(r"[,!.]" ,  '')
reviews = mcd_review['review']

In [26]:
stop_words = nltk.corpus.stopwords.words('english')
remove_stop_words = ["not", "nor"] 
stop_words = nltk.corpus.stopwords.words('english')
sentence = " ".join(stop_words)
remove_stop_words.extend(re.findall(r"\b(\w+n't|\w{3,}n)\b", sentence))
remove_stop_words
for each in remove_stop_words:
    stop_words.remove(each)
my_stopwords = ["so", "i'm", "i've" ,"i'll","really","that's","he'll", "she'll", "when", "been", "we'll", "they'll",
               "would" , 'even' , 'could' , 'may'] 
stop_words.extend(my_stopwords)
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'in', 'out', 'on', 'off', 'over', 'under', 'further', 'once', 'here', 'there', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'only', 'own', 'same', 'so', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', "should've

In [27]:
nlp = English()
tokenizer = Tokenizer(nlp.vocab)
cleaned_reviews = []
for review in mcd_review['review']:
    tokens = tokenizer(review)
    new_words = []
    for token in tokens:
        if (token.text in stop_words) == True:
            continue
        new_words.append(token.text)
    cleaned_review = " ".join(new_words)
    if len(tokenizer(cleaned_review)) < 3:
        cleaned_review = " "
    cleaned_reviews.append(cleaned_review)
    
mcd_review['clean'] = cleaned_reviews

## Clean up the stop words and clean up the reviews with less than 3 elements.

In [28]:
nlp = spacy.load("en_core_web_sm")

In [29]:
mcd_review['nlp_review'] = mcd_review['clean'].apply(lambda x: nlp(x))
mcd_review['nlp_review']

0       (not, huge, macdonald, lover, better, ones, fa...
1       (terrible, customer, service, came, 9:30pm, st...
2       (first, ", lost, ", order, actually, gave, som...
3       (see, not, one, giving, 1, star, not, -25, sta...
4       (well, macdonald, know, food, review, reflects...
                              ...                        
1520    (enjoyed, part, repeatedly, asked, right, sauc...
1521    (worst, macdonald, long, time, dirt, everywher...
1522    (craving, macdonald, seems, closest, big, fan,...
1523    (two, points, right, gate, :, 1, thuggery, kno...
1524    (wanted, grab, breakfast, one, morning, work, ...
Name: nlp_review, Length: 1525, dtype: object

In [30]:
result = []
pair = []
for i in range(len(mcd_review['nlp_review'])):
    max_simi = 0
    for j in range((i+1),len(mcd_review['nlp_review'])):
        cos_simi = mcd_review['nlp_review'][i].similarity(mcd_review['nlp_review'][j])
        if (cos_simi > max_simi) & (cos_simi < 0.99):
            max_simi = cos_simi
            pair_max = [i,j]
    result.append(max_simi)
    pair.append(pair_max)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  


In [31]:
result

[0.8621989217718908,
 0.9115396436690267,
 0.924136024456119,
 0.6422144044566914,
 0.932900777156394,
 0.8759543947034448,
 0.942150253737846,
 0.8912123527406389,
 0.9465620564368024,
 0.938219050061854,
 0.9422890407852071,
 0.9281661050084306,
 0.9568781601954796,
 0.9246775964845924,
 0.8544339774662885,
 0.8052090624975229,
 0.8652852520385924,
 0.8526119849942638,
 0.9325568521633459,
 0.8558044071778608,
 0.9140992574002214,
 0.9526042283284065,
 0.9399600361612748,
 0.9028773111252866,
 0.9255377935980866,
 0.8602396502243306,
 0.9147350808113301,
 0.9476791284442903,
 0.8169986334787053,
 0.8777966768281186,
 0.943202918333796,
 0.7742351625770797,
 0.9083546457360427,
 0.9373132848965398,
 0.9341253552659061,
 0.8153806180878975,
 0.8619325336343295,
 0.8577022516166742,
 0.8442683143949763,
 0.9693800317857079,
 0.8545105528284136,
 0.9208283628578385,
 0.8023870307527575,
 0.9083646884946581,
 0.848210641453852,
 0.8868130274561492,
 0.8331603668606252,
 0.9236275902039974

In [32]:
index_simi = np.argsort(result)[-3:]
index_simi   
## I use a code from stackoverflow to get the index for the 10 largest cosine similarity result. 
## https://stackoverflow.com/questions/6910641/how-do-i-get-indices-of-n-maximum-values-in-a-numpy-array
## I will use index to match up at last. 

array([ 39, 634, 844])

In [33]:
for each in index_simi.tolist():
    print(pair[each])

[39, 187]
[634, 1044]
[844, 1385]


In [34]:
for each in index_simi.tolist():
    review1 = mcd_review['review'][pair[each][0]]
    review2 = mcd_review['review'][pair[each][1]]
    similarity = result[each]
    print( f'Simiarlity Score {similarity}', "\n",
           f'Review 1: \n {review1}' , "\n", 
           f'Review 2:  \n {review2}', 
         "\n\n")

Simiarlity Score 0.9693800317857079 
 Review 1: 
 toxic dump in food quality and employee humanity/work efforttypically i do not eat macdonald food; yet similar to the experience of others "trapped" at northside hospital there is no other option if you are hungry outside of the hospital cafeteria hours of operation due to a relatives stay and my visits/overnights for 11 weeks~ i too found myself "trapped" and therefore forced to buy at macdonald mostly my visits were for coffee or ice cream sundaes however last night for the first time i ordered food (grilled chicken salad) i returned it ~ the meat smelled badly this did not surprise me and only enforced my resolve to avoid chain food restaurantswhat did surprise me and absolutely shocked me was the customer service offered by employees ~ how in the world can you be an employee here and not for one minute or even a second acknowledge the obvious fact that you work inside a hospital; then consider what that means for each and every pers

**Part II:**

In [142]:
train =  pd.read_csv("SMS_train.csv", encoding = 'latin-1' )
test = pd.read_csv("SMS_test.csv", encoding = 'latin-1' )

In [143]:
len(test.loc[test['Label'] == 'Spam']) / len(test) ## The Spam rate is 0.608 for the test datasets.

0.608

In [144]:
train['Label'].loc[train['Label'] == 'Non-Spam'] = 0
train['Label'].loc[train['Label'] == 'Spam'] = 1
train['Message_body'] = train['Message_body'].str.lower()

test['Label'].loc[test['Label'] == 'Non-Spam'] = 0
test['Label'].loc[test['Label'] == 'Spam'] = 1
test['Message_body'] = test['Message_body'].str.lower()


In [193]:
def clean_message(text):
    text = text.replace('*', '')
    text = re.sub(r':', r' ', text)
    text = re.sub(r'<br />', r' ', text)
    text = re.sub(r'\b(ü)\b', r'u', text)
    text = re.sub(r'\S?(www|http)\S+', r'URL', text)
    text = re.sub(r'((\d){9,})|(\d+-\d+-\d+)|((\d){3,}\s?(\d){3,}?\s?(\d){4,}?\S+?)' , r'phone_number', text)

    return text
train['Message_body'] = train['Message_body'].apply(clean_message)
test['Message_body'] = test['Message_body'].apply(clean_message)

In [205]:
nlp = spacy.load("en_core_web_md", disable=["ner", "tagger", "parser", "attribute_ruler"])
train['Message_body_nlp'] = train['Message_body'].apply(lambda x: nlp(x))
test['Message_body_nlp'] = test['Message_body'].apply(lambda x: nlp(x))

In [206]:
clean_message = []
for each in train['Message_body_nlp']:
    cleaned_doc = " ".join([token.lemma_ for token in each if not token.is_stop])
    clean_message.append(cleaned_doc)
clean_message
train['clean'] = clean_message

In [207]:
clean_message = []
for each in test['Message_body_nlp']:
    cleaned_doc = " ".join([token.lemma_ for token in each if not token.is_stop])
    clean_message.append(cleaned_doc)
clean_message
test['clean'] = clean_message

In [215]:
df = train.append(test)
df

,S. No.,Message_body,Label,Message_body_nlp,clean
0,1,rofl. its true to its name,0,"(rofl, ., its, true, to, its, name)",rofl . true
1,2,the guy did some bitching but i acted like i'd...,0,"(the, guy, did, some, bitching, but, i, acted,...",guy bitching acted like interested buying week...
2,3,"pity, was in mood for that. so...any other su...",0,"(pity, ,, , was, in, mood, for, that, ., so, ...","pity , mood . ... suggestions ?"
3,4,will u b going to esplanade fr home?,0,"(will, u, b, going, to, esplanade, fr, home, ?)",u b going esplanade fr home ?
4,5,this is the 2nd time we have tried 2 contact u...,1,"(this, is, the, 2nd, time, we, have, tried, 2,...",2nd time tried 2 contact u. u won £ 750 pound ...
...,...,...,...,...,...
120,121,7 wonders in my world 7th you 6th ur style 5th...,0,"(7, wonders, in, my, world, 7th, you, 6th, ur,...",7 wonders world 7th 6th ur style 5th ur smile ...
121,122,try to do something dear. you read something f...,0,"(try, to, do, something, dear, ., you, read, s...",try dear . read exams
122,123,sun ah... thk mayb can if dun have anythin on....,0,"(sun, ah, ..., thk, mayb, can, if, dun, have, ...",sun ah ... thk mayb dun anythin ... thk book e...
123,124,"symptoms when u are in love ""1.u like listeni...",0,"(symptoms, when, u, are, in, love, , "", 1.u, ...","symptoms u love "" 1.u like listening songs 2..."


In [269]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib
from sklearn.pipeline import Pipeline

 - **Count Vectorizer:**

In [1]:
pipe_countVectorizer = Pipeline([('countV', CountVectorizer( stop_words = "english", token_pattern = '[a-z0-9+]')),
                         ('regression', LogisticRegression())])
grid_logistic_parameter = {
    'countV__max_features': (500, 1000,1500),
    'countV__min_df': (0.001, 0.005, 0.01, 0.02),
    'countV__ngram_range':((1,1),(1,2),(1,3)),
    'regression__C': (0.1,1,10),
    'regression__solver': ( 'liblinear', 'sag', 'lbfgs')}

X = df['clean']
y = df['Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size= 957, shuffle=False)

grid_search = GridSearchCV(pipe_countVectorizer, 
                           param_grid=grid_logistic_parameter,
                           cv=10, n_jobs=-1, scoring='accuracy')
model_countV = grid_search.fit(X_train,y_train)
y_predicted_countV = model_countV.predict(X_test)

NameError: name 'Pipeline' is not defined

In [322]:
model_countV.best_params_

{'countV__max_features': 500,
 'countV__min_df': 0.001,
 'countV__ngram_range': (1, 2),
 'regression__C': 0.1,
 'regression__solver': 'liblinear'}

In [323]:
model_countV.score(X_test, y_test)

0.864

In [372]:
roc_auc_score( y_test , y_predicted_countV  )

0.888157894736842

In [376]:
confusion_matrix(y_test , y_predicted_countV)

array([[49,  0],
       [17, 59]])

In [385]:
wrongprediction = np.nonzero(y_predicted_countV != y_test)
np.take(np.array(test['clean']),wrongprediction)

array([['sms . ac blind date 4u !   rodds1 21 / m aberdeen , united kingdom . check http //img . sms . ac / w / icmb3cktz8r7!-4 blind dates send hide',
        'ringtone club   uk singles chart mobile week choose quality ringtone ! message free charge .',
        'hey horny want chat naked text hot 69698 text charged 150pm unsubscribe text stop 69698',
        'u phone_number secret admirer looking 2 contact u - find rreveal thinks ur special - phone_number',
        'text & meet sexy today . u find date flirt u. join 4 10p . reply & age eg sam 25 . 18 -msg recd@thirtyeight pence',
        'u meet ur dream partner soon ? ur career 2 flyng start ? 2 find free , txt horo followed ur star sign , e. g. horo aries',
        'unique ? find 30th august . url',
        'bangbabes ur order way . u receive service msg 2 download ur content . u , goto wap . bangb . tv ur mobile internet / service menu',
        'rcv msgs chat svc . free hardcore services text   69988 u u age verify yr network & t

 - When we look at the wrong reviews above from the Count Vectorization model, we can see that the model fails to classify corretly for spam messages invovle 'message', 'text back/txt back', 'reply'. Thus, I may need to use regex to clean them into one words so that Count Vectorization may achieve better results. 

 - **TF-IDF**

In [324]:
pipe_tfidf = Pipeline([('tfidf', TfidfVectorizer( stop_words = "english", token_pattern = '[?!a-z0-9+]')),
                         ('regression', LogisticRegression())])
tfidf_parameter = {
    'tfidf__max_features': (500, 1000,1500, 2000),
    'tfidf__min_df': (0.001, 0.005, 0.01),
    'tfidf__ngram_range':((1,1),(1,2),(1,3)),
    'regression__C': (0.1,1,10),
    'regression__solver': ( 'liblinear', 'sag', 'lbfgs')}

X = df['clean']
y = df['Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size= 957, shuffle=False)

grid_search_2 = GridSearchCV(pipe_tfidf, 
                           param_grid= tfidf_parameter,
                           cv=5, n_jobs=-1, scoring='accuracy')
model_tfidf = grid_search_2.fit(X_train,y_train)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [325]:
model_tfidf.best_params_

{'regression__C': 10,
 'regression__solver': 'lbfgs',
 'tfidf__max_features': 500,
 'tfidf__min_df': 0.001,
 'tfidf__ngram_range': (1, 3)}

In [326]:
model_tfidf.score(X_test, y_test)

0.904

In [373]:
roc_auc_score( y_test , y_predicted_tfidf  )

0.9210526315789473

In [386]:
confusion_matrix(y_test , y_predicted_tfidf)

array([[49,  0],
       [12, 64]])

In [389]:
wrongprediction = np.nonzero(y_predicted_tfidf != y_test)
np.take(np.array(test['clean']),wrongprediction)

array([['sms . ac blind date 4u !   rodds1 21 / m aberdeen , united kingdom . check http //img . sms . ac / w / icmb3cktz8r7!-4 blind dates send hide',
        'ringtone club   uk singles chart mobile week choose quality ringtone ! message free charge .',
        'text & meet sexy today . u find date flirt u. join 4 10p . reply & age eg sam 25 . 18 -msg recd@thirtyeight pence',
        'u meet ur dream partner soon ? ur career 2 flyng start ? 2 find free , txt horo followed ur star sign , e. g. horo aries',
        'unique ? find 30th august . url',
        'bangbabes ur order way . u receive service msg 2 download ur content . u , goto wap . bangb . tv ur mobile internet / service menu',
        'rcv msgs chat svc . free hardcore services text   69988 u u age verify yr network & try',
        'hear new " divorce barbie " ? comes ken stuff !',
        'sms . ac sptv   new jersey devils detroit red wings play ice hockey . correct incorrect ? end ? reply end sptv',
        'thanks subscr

- From the result above, we can see that TF-IDF appraoch fails to classify those spam with email address '@' and monetary symbols like '£' correctly. I should try to transform all the email format into one token and keep all the monetary symbols into the vector transfomration.

 - **Word2Vec**

In [344]:
train['message_vector'] = train['Message_body_nlp'].apply(lambda x: x.vector)
test['message_vector'] = test['Message_body_nlp'].apply(lambda x: x.vector)
X_train = np.array(train.message_vector.tolist())
X_test = np.array(test.message_vector.tolist())
y_train = train['Label'].values
y_test = test['Label'].values

logistic = LogisticRegression()
model_w2v = logistic.fit(X_train,y_train)
y_predicted = model_w2v.predict(X_test)
confusion_matrix(y_test, y_predicted)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


array([[49,  0],
       [17, 59]])

In [369]:
X_train = np.array(train.message_vector.tolist())
X_test = np.array(test.message_vector.tolist())
y_train = train['Label'].values
y_test = test['Label'].values
pipe_w2v = Pipeline([('regression', LogisticRegression())])
logistic_parameter = {
    'regression__C': (0.1,1,10,100),
    'regression__solver': ( 'liblinear', 'sag', 'lbfgs')}
grid_search_w2v = GridSearchCV(pipe_w2v, param_grid = logistic_parameter, cv = 5, verbose=True, n_jobs=-1)
w2v_grid_model = grid_search_w2v.fit(X_train,y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    2.7s finished


In [370]:
w2v_grid_model.best_params_

{'regression__C': 10, 'regression__solver': 'liblinear'}

In [371]:
w2v_grid_model.score(X_test, y_test)

0.912

In [374]:
y_predicted_w2v = w2v_grid_model.predict(X_test)

In [375]:
roc_auc_score( y_test , y_predicted_w2v  )

0.9240064446831364

In [390]:
confusion_matrix(y_test , y_predicted_w2v)

array([[48,  1],
       [10, 66]])

In [391]:
wrongprediction = np.nonzero(y_predicted_w2v != y_test)
np.take(np.array(test['clean']),wrongprediction)

array([['contacted dating service entered phone fancy ! find landline phone_number . pobox12n146tf150p',
        'u phone_number secret admirer looking 2 contact u - find rreveal thinks ur special - phone_number',
        'text & meet sexy today . u find date flirt u. join 4 10p . reply & age eg sam 25 . 18 -msg recd@thirtyeight pence',
        'u meet ur dream partner soon ? ur career 2 flyng start ? 2 find free , txt horo followed ur star sign , e. g. horo aries',
        'freemsg replied text ? randy , sexy , female live local . luv hear u. netcollex ltd phone_numberp msg reply stop end',
        'rcv msgs chat svc . free hardcore services text   69988 u u age verify yr network & try',
        'hear new " divorce barbie " ? comes ken stuff !',
        'sms . ac sptv   new jersey devils detroit red wings play ice hockey . correct incorrect ? end ? reply end sptv',
        'freemsg hey darling 3 week word ! like fun ? tb ok ! xxx std chgs send , £ 1.50 rcv',
        'running managed 5

 - Word to Vector Embedding after gridsearch tuning acheives the highest accuray rate and roc score.